### Pre treatment interaction dummy

In [56]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import datetime

In [57]:
nielsen15 = pd.read_csv('../../Nielsen/aggregated_nielsen_2015.csv')
nielsen16 = pd.read_csv('../..//Nielsen/aggregated_nielsen_2016.csv')
nielsen15['year'] = 2015
nielsen16['year'] = 2016

nielsen = pd.concat((nielsen15, nielsen16))
nielsen = nielsen[~nielsen.is_walmart]

#density_path = r"C:\Users\inesn\Documents\Mines\2A\TR\econometrics\data_collection\plein_de_data\Average_Household_Size_and_Population_Density_-_County.csv"
#density = pd.read_csv(density_path)
#density = density[["GEOID", "B01001_calc_PopDensity"]]
#nielsen = pd.merge(nielsen, density,  how='inner', left_on='guessed_store_county_fips', right_on='GEOID')
#nielsen = nielsen.rename(columns={"B01001_calc_PopDensity": "Population density (people/km^2)"})

## Entriy/exit dates
fandom = pd.read_csv('../data_collection/plein_de_data/fandom_traitées.csv', parse_dates=['Opening_date', 'Closing_date'])[['State', 'County_name', 'County_fips', 'Opening_date', 'Closing_date']]

# We drop the state in which we do not trust our data (some mistakes stillremain)
fandom = fandom[~np.isin(fandom.State, ('CA', 'GA', 'KS', 'LA', 'TX'))]
nielsen = nielsen[~np.isin(nielsen.store_state, ('CA', 'GA', 'KS', 'LA', 'TX'))]

# We concentrate our study on the movements (entries & exits) during the fiscal years 2015 and 2016
#movements = fandom[((fandom.Opening_date >= '2015-01-31') & (fandom.Opening_date <= '2017-01-31')) | ((fandom.Closing_date >= '2015-01-31') & (fandom.Closing_date <= '2017-01-31'))]
# Correction des effets de bord M12
movements = fandom[((fandom.Opening_date >= '2014-01-31') & (fandom.Opening_date <= '2018-01-31')) | ((fandom.Closing_date >= '2014-01-31') & (fandom.Closing_date <= '2018-01-31'))]

In [60]:
categories = [
    "BEER",
    "BREAKFAST FOODS - FROZEN",
    "CANDY",
    "CONDIMENTS, GRAVIES, AND SAUCES",
    "COOKIES",
    "COT CHEESE, SOUR CREAM, TOPPINGS",
    "COUGH AND COLD REMEDIES", #<-pb
    "DEODORANT",
    "DESSERTS, GELATINS, SYRUP",
    "DESSERTS/FRUITS/TOPPINGS-FROZEN",
    "DISPOSABLE DIAPERS",
    "DRESSINGS/SALADS/PREP FOODS-DELI",
    "GRT CARDS/PARTY NEEDS/NOVELTIES",
    "ICE CREAM, NOVELTIES",
    "JAMS, JELLIES, SPREADS",
    "JUICE, DRINKS - CANNED, BOTTLED",
    "MILK",
    "PAPER PRODUCTS",
    "PIZZA/SNACKS/HORS DOEUVRES-FRZN",
    "PREPARED FOOD-DRY MIXES",
    "PREPARED FOOD-READY-TO-SERVE", 
    "SEAFOOD - CANNED",
    "SNACKS",
    "SPICES, SEASONINGS, EXTRACTS",
    "UNPREP MEAT/POULTRY/SEAFOOD-FRZN",
    "VEGETABLES - CANNED", 
    "VEGETABLES-FROZEN",
    "WRAPPING MATERIALS AND BAGS",
]

In [61]:
# We choose to focus on milk prices
for category in categories :
    product_group = nielsen[nielsen.product_group_descr == category]
    #product_group = pool
    #product_group = nielsen

    # The control group is composed by all states where nothing (no entry nor exit) happened.
    control = product_group[~np.isin(product_group.guessed_store_county_fips, movements)].copy()
    print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

    # We keep in the control group the only counties where we have data for the entire time period (24 months)
    nb_months = control.groupby('guessed_store_county_fips').count()
    control = control[np.isin(control.guessed_store_county_fips, nb_months[nb_months.is_walmart==24].index)]
    print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

    nb_obs = control.groupby('guessed_store_county_fips').min()
    control = control[np.isin(control.guessed_store_county_fips, nb_obs[nb_obs.nb_of_obs > 3].index)]
    print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

    # The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
    count = movements.groupby('County_fips').count()
    count = count[count.State == 1] # No more than one movement in the treatement group
    treatment_movements = movements[(np.isin(movements.County_fips, count.index))]
    treatment_movements = treatment_movements[(treatment_movements.Opening_date>='2015-01-31' ) & (treatment_movements.Opening_date<='2017-01-31') & ((treatment_movements.Closing_date>'2017-01-31') | (treatment_movements.Closing_date.apply(str) == 'NaT'))]

    treatment = product_group[np.isin(product_group.guessed_store_county_fips, treatment_movements.County_fips )].copy()
    treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
    print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")



    # We create our dummies for the regression
    control['treat'] = False
    control['interaction'] = False
    control['purchase_0'] = control.purchase_month + 12 * (control.purchase_year - 2015)
    control['effects'] = list(zip(control.store_state, control.purchase_0))
    control['p_interaction'] = False

    treatment['treat'] = True
    treatment['purchase_0'] = treatment.purchase_month + 12 * (treatment.purchase_year - 2015)
    treatment['opening_0'] = treatment.Opening_date.dt.month  + 12 * (treatment.Opening_date.dt.year - 2015)
    treatment['interaction'] = treatment.purchase_0 >= treatment.opening_0
    treatment['effects'] = list(zip(treatment.store_state, treatment.purchase_0))
    treatment['p_interaction'] = treatment.purchase_0 < treatment.opening_0
    
    treatment = treatment[abs(treatment.purchase_0-treatment.opening_0) >= 6]
    
    if treatment.shape[0]>0 :
        # Final dataset for the regression :

        df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction', 'effects', "p_interaction"]]
        df = df[df.upc_price != 0]

        reg1 = smf.ols(formula='np.log(upc_price) ~ treat + interaction + p_interaction + C(effects)', data=df)
        results1 = reg1.fit()
        print("=========================================================")
        print(category)
        if abs(results1.params[2] / results1.bse[2]) >= 2.:
            print(f"Coef : {np.exp(results1.params[2])-1}")
        print(f"Coef/err : {abs(results1.params[2] / results1.bse[2])}")
        if abs(results1.params[2] / results1.bse[2]) >= 2.:
            print(np.exp(results1.params[3])-1)
            print(np.abs(results1.params[3] / results1.bse[3]))
            print(f"CI_up : {np.exp(results1.conf_int(alpha=0.05)[0][2])-1}")
            print(f"CI_down : {np.exp(results1.conf_int(alpha=0.05)[1][2])-1}")

Size of the control group: 1223.
Size of the control group: 205.
Size of the control group: 96.
Size of the treatment group: 61.
BEER
Coef/err : 1.794516320352752
Size of the control group: 0.
Size of the control group: 0.
Size of the control group: 0.
Size of the treatment group: 0.
Size of the control group: 1751.
Size of the control group: 701.
Size of the control group: 460.
Size of the treatment group: 65.
CANDY
Coef/err : 1.9352949488546434
Size of the control group: 1751.
Size of the control group: 751.
Size of the control group: 485.
Size of the treatment group: 65.
CONDIMENTS, GRAVIES, AND SAUCES
Coef/err : 0.7437432532587933
Size of the control group: 1692.
Size of the control group: 532.
Size of the control group: 316.
Size of the treatment group: 65.
COOKIES
Coef : 0.027910347859738227
Coef/err : 3.677330351525849
-0.039800797170626456
5.847321867430004
CI_up : 0.012936515237143453
CI_down : 0.04310553262043526
Size of the control group: 1692.
Size of the control group: 579

### Pooled results

In [40]:
pool = nielsen.groupby(['store_state','guessed_store_county','guessed_store_county_fips','purchase_month','purchase_year']).mean().reset_index()
product_group = pool
#product_group = nielsen

# The control group is composed by all states where nothing (no entry nor exit) happened.
control = product_group[~np.isin(product_group.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

# We keep in the control group the only counties where we have data for the entire time period (24 months)
nb_months = control.groupby('guessed_store_county_fips').count()
control = control[np.isin(control.guessed_store_county_fips, nb_months[nb_months.is_walmart==24].index)]
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

#nb_obs = control.groupby('guessed_store_county_fips').min()
#control = control[np.isin(control.guessed_store_county_fips, nb_obs[nb_obs.nb_of_obs > 3].index)]
#print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.State == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]
treatment_movements = treatment_movements[(treatment_movements.Opening_date>='2015-01-31' ) & (treatment_movements.Opening_date<='2017-01-31') & ((treatment_movements.Closing_date>'2017-01-31') | (treatment_movements.Closing_date.apply(str) == 'NaT'))]

treatment = product_group[np.isin(product_group.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")

# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
control['purchase_0'] = control.purchase_month + 12 * (control.purchase_year - 2015)
control['effects'] = list(zip(control.store_state, control.purchase_0))
control['p_interaction'] = False

treatment['treat'] = True
treatment['purchase_0'] = treatment.purchase_month + 12 * (treatment.purchase_year - 2015)
treatment['opening_0'] = treatment.Opening_date.dt.month  + 12 * (treatment.Opening_date.dt.year - 2015)
treatment['interaction'] = treatment.purchase_0 >= treatment.opening_0
treatment['effects'] = list(zip(treatment.store_state, treatment.purchase_0))
treatment['p_interaction'] = treatment.purchase_0 < treatment.opening_0

treatment = treatment[abs(treatment.purchase_0-treatment.opening_0) >= 6]

if treatment.shape[0]>0 :
    # Final dataset for the regression :

    df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction', 'effects', "p_interaction"]]
    df = df[df.upc_price != 0]

    reg1 = smf.ols(formula='np.log(upc_price) ~ treat + interaction + p_interaction + C(effects)', data=df)
    results1 = reg1.fit()
    print("=========================================================")
    if abs(results1.params[2] / results1.bse[2]) >= 2.:
        print(f"Post-t interaction Coef : {np.exp(results1.params[2])-1}")
    print(f"Coef/err : {abs(results1.params[2] / results1.bse[2])}")
    if abs(results1.params[2] / results1.bse[2]) >= 2.:
        print(f"CI_up : {np.exp(results1.conf_int(alpha=0.05)[0][2])-1}")
        print(f"CI_down : {np.exp(results1.conf_int(alpha=0.05)[1][2])-1}")
        print(f"Pre-t interaction Coef : {np.exp(results1.params[3])-1}")
        print(f"Coef/err : {abs(results1.params[3] / results1.bse[3])}")

C:\Users\inesn\AppData\Local\Temp\ipykernel_41112\1472247794.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pool = nielsen.groupby(['store_state','guessed_store_county','guessed_store_county_fips','purchase_month','purchase_year']).mean().reset_index()


Size of the control group: 1802.
Size of the control group: 1390.
Size of the treatment group: 65.
Post-t interaction Coef : 0.0706284805014521
Coef/err : 7.499050932256935
CI_up : 0.05170046901642289
CI_down : 0.08989714945439364
Pre-t interaction Coef : 0.017884479832837474
Coef/err : 2.0847475037421326


In [ ]:
#control['p_interaction'] = False
#treatment['p_interaction'] = treatment.purchase_0 < treatment.opening_0
#df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction', 'store_state', 'time_effects', 'p_interaction']]

### Placebo 1 - placebo treatment group

We aim at designing a false treatment group. Each element of the actual treatment group will be replaced by a randomly selected element from the control group that is located in the same state. We will run our previously used model on this new "control group" and compare the statistical significance of the interaction coefficient with the one obtained using our original treatment group. 

In [13]:
#TO_ADD
from random import choice
#END_TO_ADD

In [14]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import datetime


nielsen15 = pd.read_csv('../../Nielsen/aggregated_nielsen_2015.csv')
nielsen16 = pd.read_csv('../..//Nielsen/aggregated_nielsen_2016.csv')
nielsen15['year'] = 2015
nielsen16['year'] = 2016

nielsen = pd.concat((nielsen15, nielsen16))
nielsen = nielsen[~nielsen.is_walmart]

## Entriy/exit dates
fandom = pd.read_csv('../data_collection/plein_de_data/fandom_traitées.csv', parse_dates=['Opening_date', 'Closing_date'])[['State', 'County_name', 'County_fips', 'Opening_date', 'Closing_date']]

# We drop the state in which we do not trust our data (some mistakes stillremain)
fandom = fandom[~np.isin(fandom.State, ('CA', 'GA', 'KS', 'LA', 'TX'))]
nielsen = nielsen[~np.isin(nielsen.store_state, ('CA', 'GA', 'KS', 'LA', 'TX'))]

# We concentrate our study on the movements (entries & exits) during the fiscal years 2015 and 2016
#movements = fandom[((fandom.Opening_date >= '2015-01-31') & (fandom.Opening_date <= '2017-01-31')) | ((fandom.Closing_date >= '2015-01-31') & (fandom.Closing_date <= '2017-01-31'))]
# Correction des effets de bord M12
movements = fandom[((fandom.Opening_date >= '2014-01-31') & (fandom.Opening_date <= '2018-01-31')) | ((fandom.Closing_date >= '2014-01-31') & (fandom.Closing_date <= '2018-01-31'))]

In [15]:
# We choose to focus on milk prices
category = 'MILK'
product_group = nielsen[nielsen.product_group_descr == category]

# The control group is composed by all states where nothing (no entry nor exit) happened.
control = product_group[~np.isin(product_group.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

# We keep in the control group the only counties where we have data for the entire time period (24 months)
nb_months = control.groupby('guessed_store_county_fips').count()
control = control[np.isin(control.guessed_store_county_fips, nb_months[nb_months.is_walmart==24].index)]
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

nb_obs = control.groupby('guessed_store_county_fips').min()
control = control[np.isin(control.guessed_store_county_fips, nb_obs[nb_obs.nb_of_obs > 3].index)]
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.State == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]
treatment_movements = treatment_movements[(treatment_movements.Opening_date>='2015-01-31' ) & (treatment_movements.Opening_date<='2017-01-31') & ((treatment_movements.Closing_date>'2017-01-31') | (treatment_movements.Closing_date.apply(str) == 'NaT'))]

treatment = product_group[np.isin(product_group.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")

Size of the control group: 1753.
Size of the control group: 974.
Size of the control group: 660.
Size of the treatment group: 65.


In [23]:
treatment.dtypes

is_walmart                             bool
store_state                          object
guessed_store_county                 object
guessed_store_county_fips             int64
purchase_year                         int64
purchase_month                        int64
product_group_descr                  object
upc_price                           float64
upc_price_std                       float64
nb_of_obs                             int64
year                                  int64
State                                object
County_name                          object
County_fips                           int64
Opening_date                 datetime64[ns]
Closing_date                 datetime64[ns]
dtype: object

In [16]:
#TO_ADD

#We create an 'Opening_date' column for the control group (because we have to add an Opening_date to the
#placebo treatment group counties)

control['Opening_date']=0

#We gather relevant information on the actual treatment group in an array

treatment_info = treatment.groupby(['guessed_store_county_fips','store_state','Opening_date']).nunique()['is_walmart'].reset_index().to_numpy()


#We select our placebo treatment group counties

new_t_county_fips_list = []
#total nb of counties we want in the placebo treatment group
nb_of_t_counties = treatment_info.shape[0]
#list of indexes in treatment_info of the actual Tg counties for which we could not find a placebo in the same state
index_of_missing_counties = []
for k in range(nb_of_t_counties): 
    #for each actual Tg county, we find a placebo replacement
    #we get the county state
    state = treatment_info[k][1]
    #we get the county opening date
    opening_date = treatment_info[k][2] 
    #list of the candidate county_fips for our placebo (same state as the actual Tg county, not the same county as another already existing placebo county)
    c_county_list = control[(control.store_state == state)&(~np.isin(control.guessed_store_county_fips, new_t_county_fips_list))].guessed_store_county_fips.unique().copy().tolist() 
    nb_of_c_counties = len(c_county_list) #nb of candidates
    if nb_of_c_counties > 0 :
        new_t_county_fips = choice(c_county_list) #random choice of one candidate
        new_t_county_fips_list.append(new_t_county_fips)
        #we add the right opening_date to placebo county
        control.loc[control.guessed_store_county_fips==new_t_county_fips, 'Opening_date'] = opening_date 
    else : 
        #if we did not find suitable candidates in the right state
        index_of_missing_counties.append(k)
        
nb_of_missing_counties = len(index_of_missing_counties)
if nb_of_missing_counties != 0 : 
    #if we did not find suitable candidates in the right state for at least one state, we search candidates whatever the state
    for k in index_of_missing_counties:
        remaining_c_county_list = control[~np.isin(control.guessed_store_county_fips, new_t_county_fips_list)].guessed_store_county_fips.unique().copy().tolist()
        new_t_county_fips = choice(remaining_c_county_list)
        new_t_county_fips_list.append(new_t_county_fips)
        control.loc[control.guessed_store_county_fips==new_t_county_fips, 'Opening_date'] = opening_date

#we keep in our placebo Tg only the selected placebo counties
new_treatment = control[np.isin(control.guessed_store_county_fips, new_t_county_fips_list)] 
#END_TO_ADD

#DON'T FORGET TO CHANGE names : treatment -> new_treatment bellow

In [29]:
treatment_info[0][2].month

11

C:\Users\inesn\AppData\Local\Temp\ipykernel_1744\2299134106.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control[control.guessed_store_county_fips==new_t_county_fips]['Opening_date'] = opening_date


In [90]:
pd.set_option('display.max_rows', 5000)
treatment.head()

,is_walmart,store_state,guessed_store_county,guessed_store_county_fips,purchase_year,purchase_month,product_group_descr,upc_price,upc_price_std,nb_of_obs,year,State,County_name,County_fips,Opening_date,Closing_date
0,False,AL,CULLMAN,1043,2015,1,MILK,3.225000,0.609818,24,2015,AL,Cullman,1043,2015-11-11,NaT
1,False,AL,CULLMAN,1043,2015,2,MILK,3.120000,0.932845,27,2015,AL,Cullman,1043,2015-11-11,NaT
2,False,AL,CULLMAN,1043,2015,3,MILK,3.264545,1.061871,33,2015,AL,Cullman,1043,2015-11-11,NaT
3,False,AL,CULLMAN,1043,2015,4,MILK,3.090263,0.948046,38,2015,AL,Cullman,1043,2015-11-11,NaT
4,False,AL,CULLMAN,1043,2015,5,MILK,3.084815,0.715178,27,2015,AL,Cullman,1043,2015-11-11,NaT


In [91]:
pd.set_option('display.max_rows', 5000)
new_treatment.head()

,is_walmart,store_state,guessed_store_county,guessed_store_county_fips,purchase_year,purchase_month,product_group_descr,upc_price,upc_price_std,nb_of_obs,year,Opening_date
26,False,AL,AUTAUGA,1001,2015,1,MILK,3.572727,0.338942,11,2015,2015-09-30 00:00:00
68,False,AL,AUTAUGA,1001,2015,2,MILK,3.490000,0.339074,8,2015,2015-09-30 00:00:00
100,False,AL,AUTAUGA,1001,2015,3,MILK,3.911667,0.090646,6,2015,2015-09-30 00:00:00
146,False,AL,AUTAUGA,1001,2015,4,MILK,3.586000,0.795695,5,2015,2015-09-30 00:00:00
186,False,AL,AUTAUGA,1001,2015,5,MILK,3.606250,0.414899,8,2015,2015-09-30 00:00:00


#### Test

In [46]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import datetime
from random import choice


nielsen15 = pd.read_csv('../../Nielsen/aggregated_nielsen_2015.csv')
nielsen16 = pd.read_csv('../..//Nielsen/aggregated_nielsen_2016.csv')
nielsen15['year'] = 2015
nielsen16['year'] = 2016

nielsen = pd.concat((nielsen15, nielsen16))
nielsen = nielsen[~nielsen.is_walmart]

## Entriy/exit dates
fandom = pd.read_csv('../data_collection/plein_de_data/fandom_traitées.csv', parse_dates=['Opening_date', 'Closing_date'])[['State', 'County_name', 'County_fips', 'Opening_date', 'Closing_date']]

# We drop the state in which we do not trust our data (some mistakes stillremain)
fandom = fandom[~np.isin(fandom.State, ('CA', 'GA', 'KS', 'LA', 'TX'))]
nielsen = nielsen[~np.isin(nielsen.store_state, ('CA', 'GA', 'KS', 'LA', 'TX'))]

# We concentrate our study on the movements (entries & exits) during the fiscal years 2015 and 2016
#movements = fandom[((fandom.Opening_date >= '2015-01-31') & (fandom.Opening_date <= '2017-01-31')) | ((fandom.Closing_date >= '2015-01-31') & (fandom.Closing_date <= '2017-01-31'))]
# Correction des effets de bord M12
movements = fandom[((fandom.Opening_date >= '2014-01-31') & (fandom.Opening_date <= '2018-01-31')) | ((fandom.Closing_date >= '2014-01-31') & (fandom.Closing_date <= '2018-01-31'))]

In [47]:
categories = [
    "BEER",
    "BREAKFAST FOODS - FROZEN",
    "CANDY",
    "CONDIMENTS, GRAVIES, AND SAUCES",
    "COOKIES",
    "COT CHEESE, SOUR CREAM, TOPPINGS",
    "COUGH AND COLD REMEDIES",
    "DEODORANT",
    "DESSERTS, GELATINS, SYRUP",
    "DESSERTS/FRUITS/TOPPINGS-FROZEN",
    "DISPOSABLE DIAPERS",
    "DRESSINGS/SALADS/PREP FOODS-DELI",
    "GRT CARDS/PARTY NEEDS/NOVELTIES",
    "ICE CREAM, NOVELTIES",
    "JAMS, JELLIES, SPREADS",
    "JUICE, DRINKS - CANNED, BOTTLED",
    "MILK",
    "PAPER PRODUCTS",
    "PIZZA/SNACKS/HORS DOEUVRES-FRZN",
    "PREPARED FOOD-DRY MIXES",
    "PREPARED FOOD-READY-TO-SERVE", #<- pb
    "SEAFOOD - CANNED",
    "SNACKS",
    "SPICES, SEASONINGS, EXTRACTS",
    "UNPREP MEAT/POULTRY/SEAFOOD-FRZN",
    "VEGETABLES - CANNED", #<- pb
    "VEGETABLES-FROZEN",
    "WRAPPING MATERIALS AND BAGS",
]

In [48]:
for category in categories :
    product_group = nielsen[nielsen.product_group_descr == category]

    # The control group is composed by all states where nothing (no entry nor exit) happened.
    control = product_group[~np.isin(product_group.guessed_store_county_fips, movements)].copy()
    print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

    # We keep in the control group the only counties where we have data for the entire time period (24 months)
    nb_months = control.groupby('guessed_store_county_fips').count()
    control = control[np.isin(control.guessed_store_county_fips, nb_months[nb_months.is_walmart==24].index)]
    print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

    nb_obs = control.groupby('guessed_store_county_fips').min()
    control = control[np.isin(control.guessed_store_county_fips, nb_obs[nb_obs.nb_of_obs > 3].index)]
    print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

    # The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
    count = movements.groupby('County_fips').count()
    count = count[count.State == 1] # No more than one movement in the treatement group
    treatment_movements = movements[(np.isin(movements.County_fips, count.index))]
    treatment_movements = treatment_movements[(treatment_movements.Opening_date>='2015-01-31' ) & (treatment_movements.Opening_date<='2017-01-31') & ((treatment_movements.Closing_date>'2017-01-31') | (treatment_movements.Closing_date.apply(str) == 'NaT'))]

    treatment = product_group[np.isin(product_group.guessed_store_county_fips, treatment_movements.County_fips )].copy()
    treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
    print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")
    
    if treatment.shape[0] > 0 : 
    
        ### Creation of the placebo treatment group

            #We create an 'Opening_date' column for the control group (because we have to add an Opening_date to the
        #placebo treatment group counties)

        control['Opening_date']=0

        #We gather relevant information on the actual treatment group in an array

        treatment_info = treatment.groupby(['guessed_store_county_fips','store_state','Opening_date']).nunique()['is_walmart'].reset_index().to_numpy()


        #We select our placebo treatment group counties

        new_t_county_fips_list = []
        error = False
        #total nb of counties we want in the placebo treatment group
        nb_of_t_counties = treatment_info.shape[0]
        #list of indexes in treatment_info of the actual Tg counties for which we could not find a placebo in the same state
        index_of_missing_counties = []
        for k in range(nb_of_t_counties): 
            #for each actual Tg county, we find a placebo replacement
            #we get the county state
            state = treatment_info[k][1]
            #we get the county opening date
            opening_date = treatment_info[k][2]
            #list of the candidate county_fips for our placebo (same state as the actual Tg county, not the same county as another already existing placebo county)
            c_county_list = control[(control.store_state == state)&(~np.isin(control.guessed_store_county_fips, new_t_county_fips_list))].guessed_store_county_fips.unique().copy().tolist() 
            nb_of_c_counties = len(c_county_list) #nb of candidates
            if nb_of_c_counties > 0 :
                new_t_county_fips = choice(c_county_list) #random choice of one candidate
                new_t_county_fips_list.append(new_t_county_fips)
                #we add the right opening_date to placebo county
                control.loc[control.guessed_store_county_fips==new_t_county_fips, 'Opening_date'] = opening_date 
            else : 
                #if we did not find suitable candidates in the right state
                index_of_missing_counties.append(k)

        nb_of_missing_counties = len(index_of_missing_counties)
        if nb_of_missing_counties != 0 : 
            #if we did not find suitable candidates in the right state for at least one state, we search candidates whatever the state
            for k in index_of_missing_counties:
                remaining_c_county_list = control[~np.isin(control.guessed_store_county_fips, new_t_county_fips_list)].guessed_store_county_fips.unique().copy().tolist()
                print(f'remaining_c_county_list len : {len(remaining_c_county_list)}')
                if len(remaining_c_county_list) <= 0 :
                    error = True
                else :
                    new_t_county_fips = choice(remaining_c_county_list)
                    new_t_county_fips_list.append(new_t_county_fips)
                    control.loc[control.guessed_store_county_fips==new_t_county_fips, 'Opening_date'] = opening_date
        if not error :

            #we keep in our placebo Tg only the selected placebo counties
            new_treatment = control[np.isin(control.guessed_store_county_fips, new_t_county_fips_list)].copy()
            new_control = control[~np.isin(control.guessed_store_county_fips, new_t_county_fips_list)].copy()

            ### End of the creation of the placebo treatment group

            # We create our dummies for the regression
            new_control['treat'] = False
            new_control['interaction'] = False
            new_control['purchase_0'] = new_control.purchase_month + 12 * (new_control.purchase_year - 2015)
            new_control['effects'] = list(zip(new_control.store_state, new_control.purchase_0))

            new_treatment['treat'] = True
            new_treatment['purchase_0'] = new_treatment.purchase_month + 12 * (new_treatment.purchase_year - 2015)
            new_treatment['opening_0'] = pd.DatetimeIndex(new_treatment.Opening_date).month  + 12 * (pd.DatetimeIndex(new_treatment.Opening_date).year - 2015)
            new_treatment['interaction'] = new_treatment.purchase_0 >= new_treatment.opening_0
            new_treatment['effects'] = list(zip(new_treatment.store_state, new_treatment.purchase_0))

            new_treatment = new_treatment[abs(new_treatment.purchase_0-new_treatment.opening_0) >= 6]

            if new_treatment.shape[0]>0 :
                # Final dataset for the regression :

                df = pd.concat((new_control, new_treatment))[['upc_price', 'treat', 'interaction', 'effects']]
                df = df[df.upc_price != 0]

                reg1 = smf.ols(formula='np.log(upc_price) ~ treat + interaction + C(effects)', data=df)
                results1 = reg1.fit()
                print("=========================================================")
                print(category)
                if abs(results1.params[2] / results1.bse[2]) >= 2.:
                    print(f"Coef : {np.exp(results1.params[2])-1}")
                print(f"Coef/err : {abs(results1.params[2] / results1.bse[2])}")
                if abs(results1.params[2] / results1.bse[2]) >= 2.:
                    print(f"CI_up : {np.exp(results1.conf_int(alpha=0.05)[0][2])-1}")
                    print(f"CI_down : {np.exp(results1.conf_int(alpha=0.05)[1][2])-1}")

Size of the control group: 1223.
Size of the control group: 205.
Size of the control group: 96.
Size of the treatment group: 61.
remaining_c_county_list len : 78
remaining_c_county_list len : 77
remaining_c_county_list len : 76
remaining_c_county_list len : 75
remaining_c_county_list len : 74
remaining_c_county_list len : 73
remaining_c_county_list len : 72
remaining_c_county_list len : 71
remaining_c_county_list len : 70
remaining_c_county_list len : 69
remaining_c_county_list len : 68
remaining_c_county_list len : 67
remaining_c_county_list len : 66
remaining_c_county_list len : 65
remaining_c_county_list len : 64
remaining_c_county_list len : 63
remaining_c_county_list len : 62
remaining_c_county_list len : 61
remaining_c_county_list len : 60
remaining_c_county_list len : 59
remaining_c_county_list len : 58
remaining_c_county_list len : 57
remaining_c_county_list len : 56
remaining_c_county_list len : 55
remaining_c_county_list len : 54
remaining_c_county_list len : 53
remaining_c_c

DEODORANT
Coef/err : 0.6798106764330721
Size of the control group: 1657.
Size of the control group: 379.
Size of the control group: 217.
Size of the treatment group: 65.
remaining_c_county_list len : 182
remaining_c_county_list len : 181
remaining_c_county_list len : 180
remaining_c_county_list len : 179
remaining_c_county_list len : 178
remaining_c_county_list len : 177
remaining_c_county_list len : 176
remaining_c_county_list len : 175
remaining_c_county_list len : 174
remaining_c_county_list len : 173
remaining_c_county_list len : 172
remaining_c_county_list len : 171
remaining_c_county_list len : 170
remaining_c_county_list len : 169
remaining_c_county_list len : 168
remaining_c_county_list len : 167
remaining_c_county_list len : 166
remaining_c_county_list len : 165
remaining_c_county_list len : 164
remaining_c_county_list len : 163
remaining_c_county_list len : 162
remaining_c_county_list len : 161
remaining_c_county_list len : 160
remaining_c_county_list len : 159
remaining_c_co

JAMS, JELLIES, SPREADS
Coef/err : 1.8025156274116145
Size of the control group: 1728.
Size of the control group: 698.
Size of the control group: 455.
Size of the treatment group: 64.
remaining_c_county_list len : 402
remaining_c_county_list len : 401
remaining_c_county_list len : 400
remaining_c_county_list len : 399
remaining_c_county_list len : 398
remaining_c_county_list len : 397
remaining_c_county_list len : 396
remaining_c_county_list len : 395
remaining_c_county_list len : 394
remaining_c_county_list len : 393
remaining_c_county_list len : 392
JUICE, DRINKS - CANNED, BOTTLED
Coef/err : 0.0027562157701562775
Size of the control group: 1753.
Size of the control group: 974.
Size of the control group: 660.
Size of the treatment group: 65.
remaining_c_county_list len : 597
remaining_c_county_list len : 596
MILK
Coef/err : 1.393616301290139
Size of the control group: 1699.
Size of the control group: 648.
Size of the control group: 393.
Size of the treatment group: 65.
remaining_c_coun

WRAPPING MATERIALS AND BAGS
Coef/err : 0.8698443690922716


#### Test - on poolled results

In [41]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import datetime
from random import choice


nielsen15 = pd.read_csv('../../Nielsen/aggregated_nielsen_2015.csv')
nielsen16 = pd.read_csv('../..//Nielsen/aggregated_nielsen_2016.csv')
nielsen15['year'] = 2015
nielsen16['year'] = 2016

nielsen = pd.concat((nielsen15, nielsen16))
nielsen = nielsen[~nielsen.is_walmart]

## Entriy/exit dates
fandom = pd.read_csv('../data_collection/plein_de_data/fandom_traitées.csv', parse_dates=['Opening_date', 'Closing_date'])[['State', 'County_name', 'County_fips', 'Opening_date', 'Closing_date']]

# We drop the state in which we do not trust our data (some mistakes stillremain)
fandom = fandom[~np.isin(fandom.State, ('CA', 'GA', 'KS', 'LA', 'TX'))]
nielsen = nielsen[~np.isin(nielsen.store_state, ('CA', 'GA', 'KS', 'LA', 'TX'))]

# We concentrate our study on the movements (entries & exits) during the fiscal years 2015 and 2016
#movements = fandom[((fandom.Opening_date >= '2015-01-31') & (fandom.Opening_date <= '2017-01-31')) | ((fandom.Closing_date >= '2015-01-31') & (fandom.Closing_date <= '2017-01-31'))]
# Correction des effets de bord M12
movements = fandom[((fandom.Opening_date >= '2014-01-31') & (fandom.Opening_date <= '2018-01-31')) | ((fandom.Closing_date >= '2014-01-31') & (fandom.Closing_date <= '2018-01-31'))]

In [45]:
pool = nielsen.groupby(['store_state','guessed_store_county','guessed_store_county_fips','purchase_month','purchase_year']).mean().reset_index()
product_group = pool

# The control group is composed by all states where nothing (no entry nor exit) happened.
control = product_group[~np.isin(product_group.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

# We keep in the control group the only counties where we have data for the entire time period (24 months)
nb_months = control.groupby('guessed_store_county_fips').count()
control = control[np.isin(control.guessed_store_county_fips, nb_months[nb_months.is_walmart==24].index)]
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

#nb_obs = control.groupby('guessed_store_county_fips').min()
#control = control[np.isin(control.guessed_store_county_fips, nb_obs[nb_obs.nb_of_obs > 3].index)]
#print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.State == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]
treatment_movements = treatment_movements[(treatment_movements.Opening_date>='2015-01-31' ) & (treatment_movements.Opening_date<='2017-01-31') & ((treatment_movements.Closing_date>'2017-01-31') | (treatment_movements.Closing_date.apply(str) == 'NaT'))]

treatment = product_group[np.isin(product_group.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")

if treatment.shape[0] > 0 : 

    ### Creation of the placebo treatment group

        #We create an 'Opening_date' column for the control group (because we have to add an Opening_date to the
    #placebo treatment group counties)

    control['Opening_date']=0

    #We gather relevant information on the actual treatment group in an array

    treatment_info = treatment.groupby(['guessed_store_county_fips','store_state','Opening_date']).nunique()['is_walmart'].reset_index().to_numpy()


    #We select our placebo treatment group counties

    new_t_county_fips_list = []
    error = False
    #total nb of counties we want in the placebo treatment group
    nb_of_t_counties = treatment_info.shape[0]
    #list of indexes in treatment_info of the actual Tg counties for which we could not find a placebo in the same state
    index_of_missing_counties = []
    for k in range(nb_of_t_counties): 
        #for each actual Tg county, we find a placebo replacement
        #we get the county state
        state = treatment_info[k][1]
        #we get the county opening date
        opening_date = treatment_info[k][2]
        #list of the candidate county_fips for our placebo (same state as the actual Tg county, not the same county as another already existing placebo county)
        c_county_list = control[(control.store_state == state)&(~np.isin(control.guessed_store_county_fips, new_t_county_fips_list))].guessed_store_county_fips.unique().copy().tolist() 
        nb_of_c_counties = len(c_county_list) #nb of candidates
        if nb_of_c_counties > 0 :
            new_t_county_fips = choice(c_county_list) #random choice of one candidate
            new_t_county_fips_list.append(new_t_county_fips)
            #we add the right opening_date to placebo county
            control.loc[control.guessed_store_county_fips==new_t_county_fips, 'Opening_date'] = opening_date 
        else : 
            #if we did not find suitable candidates in the right state
            index_of_missing_counties.append(k)

    nb_of_missing_counties = len(index_of_missing_counties)
    if nb_of_missing_counties != 0 : 
        #if we did not find suitable candidates in the right state for at least one state, we search candidates whatever the state
        for k in index_of_missing_counties:
            remaining_c_county_list = control[~np.isin(control.guessed_store_county_fips, new_t_county_fips_list)].guessed_store_county_fips.unique().copy().tolist()
            print(f'remaining_c_county_list len : {len(remaining_c_county_list)}')
            if len(remaining_c_county_list) <= 0 :
                error = True
            else :
                new_t_county_fips = choice(remaining_c_county_list)
                new_t_county_fips_list.append(new_t_county_fips)
                control.loc[control.guessed_store_county_fips==new_t_county_fips, 'Opening_date'] = opening_date
    if not error :

        #we keep in our placebo Tg only the selected placebo counties
        new_treatment = control[np.isin(control.guessed_store_county_fips, new_t_county_fips_list)].copy()
        new_control = control[~np.isin(control.guessed_store_county_fips, new_t_county_fips_list)].copy()

        ### End of the creation of the placebo treatment group

        # We create our dummies for the regression
        new_control['treat'] = False
        new_control['interaction'] = False
        new_control['purchase_0'] = new_control.purchase_month + 12 * (new_control.purchase_year - 2015)
        new_control['effects'] = list(zip(new_control.store_state, new_control.purchase_0))

        new_treatment['treat'] = True
        new_treatment['purchase_0'] = new_treatment.purchase_month + 12 * (new_treatment.purchase_year - 2015)
        new_treatment['opening_0'] = pd.DatetimeIndex(new_treatment.Opening_date).month  + 12 * (pd.DatetimeIndex(new_treatment.Opening_date).year - 2015)
        new_treatment['interaction'] = new_treatment.purchase_0 >= new_treatment.opening_0
        new_treatment['effects'] = list(zip(new_treatment.store_state, new_treatment.purchase_0))

        new_treatment = new_treatment[abs(new_treatment.purchase_0-new_treatment.opening_0) >= 6]

        if new_treatment.shape[0]>0 :
            # Final dataset for the regression :

            df = pd.concat((new_control, new_treatment))[['upc_price', 'treat', 'interaction', 'effects']]
            df = df[df.upc_price != 0]

            reg1 = smf.ols(formula='np.log(upc_price) ~ treat + interaction + C(effects)', data=df)
            results1 = reg1.fit()
            print("=========================================================")
            if abs(results1.params[2] / results1.bse[2]) >= 2.:
                print(f"Interaction Coef : {np.exp(results1.params[2])-1}")
            print(f"Coef/err : {abs(results1.params[2] / results1.bse[2])}")
#            if abs(results1.params[2] / results1.bse[2]) >= 2.:
#                print(f"CI_up : {np.exp(results1.conf_int(alpha=0.05)[0][2])-1}")
#                print(f"CI_down : {np.exp(results1.conf_int(alpha=0.05)[1][2])-1}")

C:\Users\inesn\AppData\Local\Temp\ipykernel_41112\3557818609.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pool = nielsen.groupby(['store_state','guessed_store_county','guessed_store_county_fips','purchase_month','purchase_year']).mean().reset_index()


Size of the control group: 1802.
Size of the control group: 1390.
Size of the treatment group: 65.
remaining_c_county_list len : 1326
Interaction Coef : -0.06451426573199459
Coef/err : 3.999849438742275


Next step : paying attention to density characterstics when choosing the placebo group ?

### Placebo 2 - a category not sold by Walmart

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import datetime

In [9]:
nielsen15 = pd.read_csv('../../Nielsen/aggregated_nielsen_2015.csv')
nielsen16 = pd.read_csv('../..//Nielsen/aggregated_nielsen_2016.csv')
nielsen15['year'] = 2015
nielsen16['year'] = 2016

nielsen2 = pd.concat((nielsen15, nielsen16))
all_cats = nielsen2["product_group_descr"].unique()

In [10]:
nielsen = nielsen2[nielsen2.is_walmart]
w_cats = nielsen["product_group_descr"].unique()

In [7]:
len(all_cats)

118

In [8]:
len(w_cats)

118

In [13]:
pd.set_option('display.max_rows', 500)
nielsen.groupby("product_group_descr").sum()

C:\Users\inesn\AppData\Local\Temp\ipykernel_1744\1431409200.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  nielsen.groupby("product_group_descr").sum()


,is_walmart,guessed_store_county_fips,purchase_year,purchase_month,upc_price,upc_price_std,nb_of_obs,year
product_group_descr,,,,,,,,
AUTOMOTIVE,21398,641501720,43127845,138614,145358.737079,59667.799787,65652,43127845
BABY FOOD,10383,312836287,20927281,66465,39761.563015,19221.847297,81582,20927281
BABY NEEDS,13749,412061104,27711330,88026,90535.395995,32880.072635,31148,27711330
BAKED GOODS-FROZEN,23158,690493295,46675148,154227,77908.758917,16343.429163,80575,46675148
BAKING MIXES,28826,870021148,58098810,188503,48033.442271,17406.832317,165773,58098810
BAKING SUPPLIES,31024,944791582,62529036,204519,60959.952322,21782.643510,215237,62529036
BATTERIES AND FLASHLIGHTS,24037,726687527,48446748,161035,169674.759351,45135.251041,76005,48446748
BEER,16120,487781967,32490024,105885,196892.211061,35984.043158,66004,32490024
BOOKS AND MAGAZINES,11693,353013614,23567022,75167,41561.860992,7633.699998,31896,23567022


In [14]:
pd.set_option('display.max_rows', 500)
nielsen2.groupby("product_group_descr").count()

,is_walmart,store_state,guessed_store_county,guessed_store_county_fips,purchase_year,purchase_month,upc_price,upc_price_std,nb_of_obs,year
product_group_descr,,,,,,,,,,
AUTOMOTIVE,27934,27934,27934,27934,27934,27934,27934,16559,27934,27934
BABY FOOD,22608,22608,22608,22608,22608,22608,22608,17566,22608,22608
BABY NEEDS,23735,23735,23735,23735,23735,23735,23735,12159,23735,23735
BAKED GOODS-FROZEN,54357,54357,54357,54357,54357,54357,54357,37859,54357,54357
BAKING MIXES,63989,63989,63989,63989,63989,63989,63989,50025,63989,63989
BAKING SUPPLIES,67507,67507,67507,67507,67507,67507,67507,54392,67507,67507
BATTERIES AND FLASHLIGHTS,43508,43508,43508,43508,43508,43508,43508,28037,43508,43508
BEER,39203,39203,39203,39203,39203,39203,39203,27847,39203,39203
BOOKS AND MAGAZINES,25627,25627,25627,25627,25627,25627,25627,15305,25627,25627
